In [1]:
import time
start_time = time.time()
from fastlane_bot import Bot, Config, ConfigDB, ConfigNetwork, ConfigProvider
from fastlane_bot.tools.cpc import ConstantProductCurve as CPC, CPCContainer, T, Pair
from fastlane_bot.tools.analyzer import CPCAnalyzer, AttrDict
from fastlane_bot.tools.optimizer import CPCArbOptimizer
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPC))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPCAnalyzer))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPCArbOptimizer))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(Bot))
from fastlane_bot.testing import *
import itertools as it
import collections as cl
plt.style.use('seaborn-dark')
plt.rcParams['figure.figsize'] = [12,6]
from fastlane_bot import __VERSION__
require("3.0", __VERSION__)
print(f"elapsed time: {time.time()-start_time:.2f}s")

Using default database url, if you want to use a different database, set the backend_url found at the bottom of manager_base.py
ConstantProductCurve v2.10.3 (07/May/2023)
CPCAnalyzer v1.0 (07/May/2023)
CPCArbOptimizer v3.6 (06/May/2023)
CarbonBot v3-b2.1 (03/May/2023)
imported m, np, pd, plt, os, sys, decimal; defined iseq, raises, require
Version = 3-b2.1 [requirements >= 3.0 is met]
elapsed time: 3.72s


# Arbitrage Dashboard [A013]

In [2]:
bot     = Bot()
CCm     = bot.get_curves()
CA      = CPCAnalyzer(CCm)
pairsc  = CA.pairsc()
print(f"elapsed time: {time.time()-start_time:.2f}s")

elapsed time: 7.57s


## All (Carbon) pairs

In [3]:
print(f"Tokens:  {len(CA.tokens()):4}")
print(f"Pairs:   {len(CA.pairs()) :4} [carbon: {len(CA.pairsc()) :4}]")
print(f"Curves:  {len(CA.curves()):4} [carbon: {len(CA.curvesc()):4}]") 

Tokens:   512
Pairs:    636 [carbon:   25]
Curves:   811 [carbon:   69]


In [4]:
for pair in CA.pairsc():
    try:
        d = CA.pair_analysis(pair)
        #print(CA.pair_analysis_pp(d))
    except Exception as e:
        pass
#         print(f"Pair:               {pair}")
#         print(f"Error:              {e}")
#         print()

## Individual pairs

In [5]:
# CODE = """
# # #### {pairn}

# pair = "{pair}"
# pricedf.loc[Pair.n(pair)]

# pi = CA.pair_data(pair)
# O = CPCArbOptimizer(pi.CC)
# r = O.margp_optimizer(pair.split("/")[0])
# r.trade_instructions(ti_format=O.TIF_DFAGGR)

# r = O.margp_optimizer(pair.split("/")[1])
# r.trade_instructions(ti_format=O.TIF_DFAGGR)
# """

# for pair, _ in curves_by_pair:
#     print(CODE.format(pairn=Pair.n(pair), pair=pair))

#### WETH/USDC

In [6]:
pair = "WETH-6Cc2/USDC-eB48"
d    = CA.pair_analysis(pair)

In [7]:
print(CA.pair_analysis_pp(d))

Pair:               WETH-6Cc2/USDC-eB48
Price:              1,911.0289
Number of curves:   20 [carbon: 16]
Value locked:       108,777,817.33 [carbon: 30,995.37, other: 108,746,821.96]
Arbitrage value:    0.09 WETH / 175.27 USDC



In [24]:
pairs1 = CCm.filter_pairs(bothin=f"{d.tknb}, {d.tknq}, {CPCContainer.TRIANGTOKENS}")
pairs1

{'BNT-FF1C/DAI-1d0F',
 'BNT-FF1C/USDC-eB48',
 'BNT-FF1C/USDT-1ec7',
 'BNT-FF1C/WBTC-C599',
 'BNT-FF1C/WETH-6Cc2',
 'DAI-1d0F/USDC-eB48',
 'DAI-1d0F/USDT-1ec7',
 'USDT-1ec7/USDC-eB48',
 'WBTC-C599/DAI-1d0F',
 'WBTC-C599/USDC-eB48',
 'WBTC-C599/USDT-1ec7',
 'WBTC-C599/WETH-6Cc2',
 'WETH-6Cc2/DAI-1d0F',
 'WETH-6Cc2/USDC-eB48',
 'WETH-6Cc2/USDT-1ec7'}

In [25]:
CCm1 = CCm.bypairs(pairs1).byparams(exchange="carbon_v1", _inv=True)
print("exchanges", {c.P("exchange") for c in CCm1})
O = CPCArbOptimizer(CCm1)
r = O.margp_optimizer(d.tknb, params=dict(verbose=False, debug=False))
ti1 = r.trade_instructions(ti_format=O.TIF_DFAGGR)
ti1

exchanges {'uniswap_v2', 'sushiswap_v2', 'uniswap_v3', 'bancor_v3', 'bancor_v2'}


In [26]:
r = O.margp_optimizer(d.tknq)
ti2 = r.trade_instructions(ti_format=O.TIF_DFAGGR)
ti2.fillna("")

,DAI-1d0F,USDC-eB48,WBTC-C599,BNT-FF1C,USDT-1ec7,WETH-6Cc2
839,31.173338,-31.193771,,,,
332,,-115.849802,0.00401,,,
624,-760.638618,,,1626.19963,,
183,,,0.00002,,-0.569817,
795,-0.287565,,,,0.286325,
840,,,,,-32507.666319,16.970429
256,,,,,-53356.415524,27.89623
290,,,,-625.799029,,0.151797
357,,-949.326251,,,947.578451,
246,,9893.050236,,,-9891.054679,


In [29]:
CCc.tokens()

{'USDC-eB48', 'WETH-6Cc2'}

In [31]:
CCm1.tokens()

{'BNT-FF1C', 'DAI-1d0F', 'USDC-eB48', 'USDT-1ec7', 'WBTC-C599', 'WETH-6Cc2'}

In [32]:
CCmc = CCm1.copy()
CCmc += CCc
print("exchanges", {c.P("exchange") for c in CCmc})
O = CPCArbOptimizer(CCmc)
r = O.margp_optimizer(d.tknb, params=dict(verbose=False, debug=False))
ti1 = r.trade_instructions(ti_format=O.TIF_DFAGGR)
ti1

exchanges {'carbon_v1', 'uniswap_v2', 'sushiswap_v2', 'uniswap_v3', 'bancor_v3', 'bancor_v2'}


In [33]:
r = O.margp_optimizer(d.tknq)
ti2 = r.trade_instructions(ti_format=O.TIF_DFAGGR)
ti2.fillna("")

,DAI-1d0F,USDC-eB48,WBTC-C599,BNT-FF1C,USDT-1ec7,WETH-6Cc2
839,31.163484,-31.183934,,,,
332,,-115.783813,0.004007,,,
624,-760.702675,,,1626.336748,,
183,,,0.00002,,-0.566138,
795,-0.28771,,,,0.28647,
840,,,,,-32499.523411,16.966164
256,,,,,-53331.51681,27.88319
290,,,,-625.782869,,0.151793
357,,-949.324254,,,947.576456,
246,,9893.070432,,,-9891.074854,


In [ ]:
O = CPCArbOptimizer(CCc)

r = O.margp_optimizer(d.tknb, params=dict(verbose=False, debug=False))
ti1 = r.trade_instructions(ti_format=O.TIF_DFAGGR)

r = O.margp_optimizer(d.tknq)
ti2 = r.trade_instructions(ti_format=O.TIF_DFAGGR)

In [ ]:
ti1

In [ ]:
ti2

In [ ]:
CPCContainer([c]).plot()

In [37]:
c=CCm.bycid("1701411834604692317316873037158841057296-0")
c.p_min, c.p_max

(1929.999806978364, 1929.9998240030798)

In [34]:
d.curvedfx

price            vl itm  bs  \
exchange     cid0                                          
carbon_v1    057285-1  2099.999790  1.268390e+01       s   
             057292-0  1853.408818  3.313530e-03       b   
             057292-1  2000.000000  3.277392e+01       s   
             057296-0  1929.999824  4.614054e-01   x   b   
             057296-1  1949.999818  1.950003e+04       s   
             057299-1  1940.000000  5.066742e+01       s   
             057306-0  1405.000140  3.558719e+00       b   
             057315-1  2300.000000  1.122285e+03       s   
             057331-0  1800.000000  5.555556e+00       b   
             057334-0  1700.000170  2.941176e-02       b   
             057334-1  1999.999800  7.999999e+01       s   
             057337-0  1850.000000  1.081081e+00       b   
             057339-0  1800.000000  5.555556e-04       b   
             057343-1  1989.999801  1.990000e+03       s   
             057353-0  1854.000185  4.234699e+00       b   
             057353-1  2047.999795  8.191999e+03       s   
sushiswap_v2 803       1920.628686  3.455222e+07      bs   
uniswap_v2   255       1915.522460  7.366285e+07      bs   
uniswap_v3   346       1910.692957  4.896770e+05   x  bs   
             593       1910.636978  4.207653e+04   x  bs   

                                                         bsv  
exchange     cid0                                             
carbon_v1    057285-1      sell-WETH @ 2100.00 USDC per WETH  
             057292-0       buy-WETH @ 1853.41 USDC per WETH  
             057292-1      sell-WETH @ 2000.00 USDC per WETH  
             057296-0       buy-WETH @ 1930.00 USDC per WETH  
             057296-1      sell-WETH @ 1950.00 USDC per WETH  
             057299-1      sell-WETH @ 1940.00 USDC per WETH  
             057306-0       buy-WETH @ 1405.00 USDC per WETH  
             057315-1      sell-WETH @ 2300.00 USDC per WETH  
             057331-0       buy-WETH @ 1800.00 USDC per WETH  
             057334-0       buy-WETH @ 1700.00 USDC per WETH  
             057334-1      sell-WETH @ 2000.00 USDC per WETH  
             057337-0       buy-WETH @ 1850.00 USDC per WETH  
             057339-0       buy-WETH @ 1800.00 USDC per WETH  
             057343-1      sell-WETH @ 1990.00 USDC per WETH  
             057353-0       buy-WETH @ 1854.00 USDC per WETH  
             057353-1      sell-WETH @ 2048.00 USDC per WETH  
sushiswap_v2 803       buy-sell-WETH @ 1920.63 USDC per WETH  
uniswap_v2   255       buy-sell-WETH @ 1915.52 USDC per WETH  
uniswap_v3   346       buy-sell-WETH @ 1910.69 USDC per WETH  
             593       buy-sell-WETH @ 1910.64 USDC per WETH

In [ ]:
d.ccurvedf

In [ ]:
d.titn

In [ ]:
d.ti1

In [ ]:
d.ti2